In [1]:
import datetime

In [3]:
MODEL_NAME = "all_leagues"
MODEL_VERSION = 1.0
MODEL_TYPE = "fulltime_win_outcome"

In [9]:
# UPDATING THE GITHUB OF THE DATASETS
MODEL_TIME = datetime.datetime.now().strftime("%d-%M-%Y_%T")
!git add -A
!git commit -m "updates: {MODEL_NAME}_{MODEL_TYPE}-v{MODEL_VERSION}: bot commit - {MODEL_TIME}"
!git push

[main 99f85f9] updates: all_leagues_fulltime_win_outcome-v1.0: bot commit - 23-19-2024_07:19:54
 1 file changed, 7 insertions(+), 5 deletions(-)
Enumerating objects: 18, done.
Counting objects: 100% (18/18), done.
Delta compression using up to 2 threads
Compressing objects: 100% (12/12), done.
Writing objects: 100% (12/12), 182.30 KiB | 758.00 KiB/s, done.
Total 12 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 4 local objects.
To https://github.com/lebyanelm/neural-trained-models.git
   808ab18..99f85f9  main -> main
